1. Класс Product

Атрибуты:
-name (строка) — название товара;
-price (число) — цена товара;
-stock (целое число) — количество товара на складе.
Методы:
-update_stock(quantity) — метод, который обновляет количество товара на складе. Если количество становится отрицательным, должно выдаваться сообщение об ошибке.

In [2]:
class NegativeStock(Exception):
    pass
class InvalidProductData(Exception):
    pass
class Product:
    def __init__(self, name:str, price:float, stock:int):
        self._validate_input_data(name, price, stock)
        self.name = name
        self.price = price
        self.stock = stock
    
    def _validate_input_data(self, name: str, price: float, stock: int):
        errors = []
        
        if not name or not isinstance(name, str):
            errors.append("Название товара должно быть непустой строкой")
        
        if not isinstance(price, (int, float)):
            errors.append("Цена должна быть числом")
        elif price <= 0:
            errors.append(f"Цена должна быть положительной. Получено: {price}")
        
        if not isinstance(stock, int):
            errors.append("Количество должно быть целым числом")
        elif stock < 0:
            errors.append(f"Количество не может быть отрицательным. Получено: {stock}")
        
        if errors:
            error_message = "Ошибки при создании товара:\n- " + "\n- ".join(errors)
            raise InvalidProductData(error_message)
        
    def update_stock(self, quantity):
        if self.stock + quantity < 0:
            raise NegativeStock(f"Количество товаров на складе не может быть отрицательным, максимальное колчество товаров для удаления - {self.stock}")
        else:
            self.stock += quantity

    def get_info(self):
        return (f"Название товара: {self.name}, цена: {self.price:.2f} руб., количество на складе: {self.stock} шт.")

    def equals_product(self, product):
        if self.name == product.name and self.price == product.price:
            return True
        else:
            return False
        
    

2. Класс Order

Атрибуты:
-products (словарь) — словарь, где ключом является объект Product, а значением — количество этого товара в заказе.
Методы:
-add_product(product, quantity) — метод для добавления товара в заказ. Если товара недостаточно на складе, должно выдаваться сообщение об ошибке;
-calculate_total() — метод для расчёта общей стоимости заказа.

In [10]:
class DificiencyProduct(Exception):
    pass
    
class Order:
    def __init__(self, store = None):
        self.products = {}
        self.store = store
        
    def add_product(self, product: Product, quantity: int):
        if product not in self.store.products:
            raise ValueError(f"Товар не продается в этом магазине. {product.get_info()}")
        if quantity <= 0:
            raise ValueError("Количество должно быть положительным")
        elif product.stock - quantity < 0:
            raise DificiencyProduct(f"Недостаточно товара на складе, максимальное колчество для добавления в заказ - {product.stock}")
        else:
            product.update_stock(-quantity)
            
        if product in self.products.keys():
            self.products[product] += quantity
        else:
            self.products[product] = quantity
            
    def return_product(self, product, quantity):
        if product not in self.store.products:
            raise ValueError(f"Товар не продается в этом магазине. {product.get_info()}")
        if quantity <= 0:
            raise ValueError("Количество должно быть положительным")
            
        if product in self.products.keys():
            product.update_stock(quantity)
            
            if self.products[product] - quantity <= 0:
                self.products.pop(product)
            else:
                self.products[product] -= quantity
        else:
            raise ValueError(f"Товар '{product.name}' отсутствует в заказе")

    def calculate_total(self):
        sum = 0.0
        for product in self.products.keys():
            sum += product.price * self.products[product]
        return sum
        
    def get_info(self):
        for i in self.products.keys():
            print(i.get_info(), ' : ' , self.products[i], ' шт. в заказе')

3. Класс Store

Атрибуты:
-products (список) — список всех доступных товаров в магазине.
Методы:
-add_product(product) — метод для добавления товара в магазин;
-list_products() — метод для отображения всех товаров в магазине с их ценами и количеством на складе;
-create_order() — метод для создания нового заказа.

In [7]:
class NegativeStock(Exception):
    pass
class Store:
    def __init__(self):
        self.products = []
        
    def add_product(self, product: Product):
        self.products.append(product)

    def list_products(self):
        if not self.products:
            print("В магазине нет товаров")
        else:
            for product in self.products:
                print(product.get_info())

    def create_order(self) -> Order:
        return Order(self)


Проверка

In [12]:
# Создаем магазин
store = Store()

# Создаем товары
product1 = Product("Ноутбук", 1000, 5)
product2 = Product("Смартфон", 500, 10)
#product3 = Product("Телевизор", 1000, 5)

# Добавляем товары в магазин
store.add_product(product1)
store.add_product(product2)

# Список всех товаров
store.list_products()

# Создаем заказ
order = store.create_order()

# Добавляем товары в заказ
order.add_product(product1, 2)
order.add_product(product2, 3)
#order.add_product(product3, 3)

# Выводим общую стоимость заказа
total1 = order.calculate_total()
print(f"Общая стоимость заказа: {total1}")

# Проверяем остатки на складе после заказа
store.list_products()

order.return_product(product2, 1)
# Выводим общую стоимость заказа
total1 = order.calculate_total()
print(f"Общая стоимость заказа: {total1}")

# Проверяем остатки на складе после заказа
store.list_products()

Название товара: Ноутбук, цена: 1000.00 руб., количество на складе: 5 шт.
Название товара: Смартфон, цена: 500.00 руб., количество на складе: 10 шт.
Общая стоимость заказа: 3500.0
Название товара: Ноутбук, цена: 1000.00 руб., количество на складе: 3 шт.
Название товара: Смартфон, цена: 500.00 руб., количество на складе: 7 шт.
Общая стоимость заказа: 3000.0
Название товара: Ноутбук, цена: 1000.00 руб., количество на складе: 3 шт.
Название товара: Смартфон, цена: 500.00 руб., количество на складе: 8 шт.
